In [ ]:
import qutip as qt
import numpy as np

sesolve.py

In [ ]:
import qutip as qt

epsilon1 = 1.0
epsilon2 = 1.0
g = 0.1

sz1 = qt.tensor(qt.sigmaz(), qt.qeye(2))
sz2 = qt.tensor(qt.qeye(2), qt.sigmaz())

sx1 = qt.tensor(qt.sigmax(), qt.qeye(2))
sx2 = qt.tensor(qt.qeye(2), qt.sigmax())

H = epsilon1 * sz1 + epsilon2 * sz2 + g * sx1 * sx2

print(H)

psi0 = qt.tensor(qt.basis(2, 0), qt.basis(2, 1))
tlist = np.linspace(0, 40, 100)
result = qt.sesolve(H, psi0, tlist)

solver = qt.SESolver(H)
result2 = solver.run(psi0, tlist)

options = {"store_states": True, "atol": 1e-12, 
"nsteps": 1000, "max_step": 0.1}
solver = qt.SESolver(H, options = options)
result2 = solver.run(psi0, tlist)
print(result2)

mesolve.py

In [ ]:
epsilon1 = 1.0
epsilon2 = 1.0
g = 0.5

sz1 = qt.tensor(qt.sigmaz(), qt.qeye(2))
sz2 = qt.tensor(qt.qeye(2), qt.sigmaz())

sx1 = qt.tensor(qt.sigmax(), qt.qeye(2))
sx2 = qt.tensor(qt.qeye(2), qt.sigmax())

H = epsilon1 * sz1 + epsilon2 * sz2 + g * sx1 * sx2


sm1 = qt.tensor(qt.destroy(2).dag(), qt.qeye(2))
sm2 = qt.tensor(qt.qeye(2), qt.destroy(2).dag())
gam = 0.1 #dissipation rate
c_ops = [np.sqrt(gam) * sm1, np.sqrt(gam) * sm2]
result_me = qt.mesolve(H, psi0, tlist, c_ops,
e_ops = [sz1, sz2])
print(result_me)


#H_L = -1.0j * (qt.spre(H)-qt.spost(H)) 
#for c in c_ops:
#    H_L += (qt.spre(c)*qt.spost(c.dag())
#    - 0.5 * (qt.spre(c.dag()*c) 
#    + qt.spost(c.dag()*c)))
#
#result_me2 = qt.mesolve(H_L, psi0, tlist, 
#            [], e_ops = [sz1, sz2])
#
#
#collapse_list = []
#all_energy, all_state = H.eigenstates()
#Nmax = len(all_state)
#tlist = np.linspace(0, 1000, 100)
#
#def sw(w):
#    if w >= 0:
#        return gam
#    else:
#        return 0
#    
#for j in range(Nmax):
#    for k in range(Nmax):
#        delE = (all_energy[k] - all_energy[j])
#        rate = np.absolute(
#        sx1.matrix_element(
#        all_state[j].dag(),all_state[k])
#        )**2 + np.absolute(
#        sx2.matrix_element(
#        all_state[j].dag(),all_state[k])
#        )**2 * sw(delE)
#        if rate > 0.0:
#            collapse_list.append(
#                (np.sqrt(2*rate) *
#                all_state[j] * 
#                all_state[k].dag())
#                )
#
#result_me_BR = qt.mesolve(H, psi0, tlist, 
#    collapse_list)
#print("Fidelity with ground-state", 
#      qt.fidelity(result_me_BR.states[-1], 
#      all_state[0] @ all_state[0].dag()))            
#
#
#def sw(w):
#    if w >= 0:
#        return gam
#    else:
#        return 0
#        
#result_BR = qt.brmesolve(H,psi0,tlist,
#                         a_ops=[[sx1,sw],
#                         [sx2,sw]], 
#                        e_ops = [sz1, sz2]
#                       )

In [ ]:
result_me = qt.mesolve(H, psi0, tlist, c_ops)

Handwaving

In [ ]:
from qutip_qoc import optimize_pulses, TimeInterval, Objective, Pulse

In [ ]:
sy1 = qt.tensor(qt.sigmay(), qt.qeye(2))
sy2 = qt.tensor(qt.qeye(2), qt.sigmay())

In [ ]:
Hc = [sx1, sx2, sy1, sy2, sz1, sz2]

In [ ]:
def sin(t, p):
    return p[0] * np.sin(p[1] * t)

def grad_sin(t, α, idx):
    if idx == 0:
        return np.sin(α[1] * t)
    if idx == 1:
        return α[0] * np.cos(α[1] * t) * t
    if idx == 2:
        return α[0] * np.cos(α[1] * t) * α[1]  # w.r.t. time

In [ ]:
H_c = [[hc, lambda t, p: sin(t, p), {"grad": grad_sin}] for hc in Hc]
initial = qt.tensor(qt.qeye(2), qt.qeye(2))
ob1 = Objective(initial,[H] + H_c, qt.gates.cnot())

In [ ]:
p_init = [1, 1]  # amplitude, frequency, phase
q_init = [1, 1]  # q[0] * sin(q[1] * t + q[2])
r_init = [1, 1]

In [ ]:
res_goat = optimize_pulses(
    objectives=[ob1],
    control_parameters={
        idx: {
            "guess": np.zeros(100),
            "bounds": [(-5, 5)]
        } for idx in range(len(H_c))
        #"__time__":{
        #    "guess": np.zeros(100),
        #    "bounds": [tlist[0], tlist[-1]]
        #}
    },
    tlist=np.linspace(0, 10, 100),
    algorithm_kwargs={
        "alg": "GRAPE",
        "fid_err_targ": 0.5,
    },
    optimizer_kwargs={ 
        "max_iter": 100, 
        "method": "basinhopping", 
        "seed": 0,
    },    
    minimizer_kwargs={
        "method": "L-BFGS-B",
        "options": {
            "maxiter": 10000,
            "gtol": 1e-8,
        },
    },
)

In [ ]:
res_goat

In [ ]:
tlist